In [2]:
## Importing library
import numpy as np
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

np.random.seed(100)

## Read file
file_name = "data/train.csv"
## Read file using pandas
df = pd.read_csv(file_name)

In [20]:
def loadEmbeddingMatrix(typeToLoad, vocab_dict):
    import gensim.models.keyedvectors as word2vec
    import gc

    # load different embedding file from Kaggle depending on which embedding
    # matrix we are going to experiment with
    if (typeToLoad == "gloveTwitter50d"):
        EMBEDDING_FILE = 'embeddings\glove-twitter-27b-50d/glove.twitter.27B.50d.txt'
        embed_size = 50
    elif (typeToLoad == "word2vec"):
        word2vecDict = word2vec.KeyedVectors.load_word2vec_format( "embeddings\GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin", binary=True)
        embed_size = 300
    elif (typeToLoad == "fasttext"):
        EMBEDDING_FILE = 'data/wiki-news-300d-1M-subword.vec'
        embed_size = 300
    elif (typeToLoad == "glove840B300D"):
        EMBEDDING_FILE = 'embeddings\glove.840B.300d/glove.840B.300d.txt'
        embed_size = 300
    elif (typeToLoad == "glove6B300D"):
        EMBEDDING_FILE = 'embeddings\glove.6B\glove.6B.300d.txt'
        embed_size = 300
    elif (typeToLoad == "paragram"):
        EMBEDDING_FILE = 'embeddings\paragram_300_sl999\paragram_300_sl999.txt'
        embed_size = 300
    elif (typeToLoad == "wikiNews"):
        EMBEDDING_FILE = "embeddings\wiki-news-300d-1M\wiki-news-300d-1M.vec"
        embed_size = 300
    elif (typeToLoad == "crawl"):
        EMBEDDING_FILE = "data\crawl-300d-2M.vec"
        embed_size = 300

    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')

    if (typeToLoad in ["gloveTwitter50d", "fasttext"]):
        embeddings_index = dict()
        # Transfer the embedding weights into a dictionary by iterating through every line of the file.
        f = open(EMBEDDING_FILE, encoding="utf-8")
        for line in f:
            # split up line into an indexed array
            values = line.rstrip().rsplit(' ')  # line.split()
            # first index is word
            word = values[0]
            # store the rest of the values in the array as a new array
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs  # 50 dimensions
        f.close()
    elif (typeToLoad in ["glove840B300D", "paragram", "glove6B300D", "crawl"]):
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='latin'))
    elif (typeToLoad in ["wikiNews"]):
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o) > 100)
    else:
        embeddings_index = dict()
        for word in word2vecDict.wv.vocab:
            embeddings_index[word] = word2vecDict.word_vec(word)
    print('Loaded %s word vectors.' % len(embeddings_index))

    gc.collect()
    # We get the mean and standard deviation of the embedding weights so that we could maintain the
    # same statistics for the rest of our own random generated weights.
    all_embs = np.stack(list(embeddings_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()

    nb_words = len(vocab_dict)
    # We are going to set the embedding size to the pretrained dimension as we are replicating it.
    # the size will be Number of Words in Vocab X Embedding Size
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    gc.collect()

    # With the newly created embedding matrix, we'll fill it up with the words that we have in both
    # our own dictionary and loaded pretrained embedding.
    embeddedCount = 0
    for word, i in vocab_dict.items():
        #i -= 1
        # then we see if this word is in glove's dictionary, if yes, get the corresponding weights
        embedding_vector = embeddings_index.get(word)
        # and store inside the embedding matrix that we will train later on.
        if embedding_vector is not None:
            try :
                embedding_matrix[i] = embedding_vector
                embeddedCount += 1
            except IndexError:
                pass
    print('total embedded:', embeddedCount, 'common words')

    del embeddings_index
    gc.collect()

    # finally, return the embedding matrix
    return embedding_matrix

In [21]:
## Tokenizing sentence into token for finding synonym.
def make_tokenizer(texts):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(texts)
    return t

tokenizer = make_tokenizer(df['description'])

In [27]:
## Dictionary of word index
index_word = {}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word

vocab_dict = tokenizer.word_index

In [29]:
tokenizer

In [23]:
## Loading word embedding
from time import time
start = time()
embed_mat = loadEmbeddingMatrix("fasttext", vocab_dict)
end = time()
print("Embedding loaded in ", (end-start)/60, "min")

Loaded 999995 word vectors.


ValueError: all input arrays must have the same shape

In [34]:
max_features = 30000
embed_size = 300
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
EMBEDDING_FILE = 'data/crawl-300d-2M.vec'
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' '))
                        for o in open(EMBEDDING_FILE, encoding="utf-8"))
word_index = tokenizer.word_index
nb_words = max(max_features, len(word_index) + 1)
print(nb_words)
embedding_matrix = np.zeros((nb_words, embed_size))
print(embedding_matrix.shape)
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.size)  # (30000, 300)

30000
(30000, 300)
9000000


In [35]:
embed_mat = embedding_matrix

In [36]:
from sklearn.neighbors import NearestNeighbors

synonyms_number = 5
word_number = 20000

nn = NearestNeighbors(n_neighbors=synonyms_number+1).fit(embed_mat)

neighbours_mat = nn.kneighbors(embed_mat[1:word_number])[1]

synonyms = {x[0]: x[1:] for x in neighbours_mat}

In [37]:
## Finding nearby synonym - Basically it's not actually synonym. It's near by words of targetted word. 
import nltk
from nltk.corpus import wordnet

synonym = {}
for x in range(0,100):
    try :
        synonym.update({index_word[x] : [index_word[synonyms[x][i]] for i in range(synonyms_number-1)]})
    except :
        pass

In [38]:
## Use this synonym list to replace words with it's variation
## Below code is in draft. But logic can be used to complete the task

In [44]:
## Can only change words for selected part of speech to preserve semantic meaning.

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download("stopwords")
nltk.download("wordnet")
def get_pos_tag (word, tagged) :
    res = [(x, y) for x, y in tagged if x == word]
    return res[0][1]
# Load the pretrained neural net
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Error loading en-ptb: Package 'en-ptb' not found in index


In [49]:

for message in df["description"]:
    print(message)
    # Tokenize the text
    tokenized = tokenizer.tokenize(message)

    # Get the list of words from the entire text
    words = word_tokenize(message)

    # Identify the parts of speech
    tagged = nltk.pos_tag(words, tagset="universal")
    
    replacements = []

    for word in words:
        synonym = []
        antonyms = []
        word_index = vocab_dict.get(word, None)

        pos_tag = get_pos_tag(word, tagged)
        if (word_index and pos_tag in ["ADJ", "ADV", "NOUN", "VERB"] and word not in nltk.corpus.stopwords.words('english')) :
            for syn in wordnet.synsets(word, eval("wordnet." + pos_tag)):
                for l in syn.lemmas() :
                    if(l.name() in [index_word[synonyms[word_index][i]] for i in range(synonyms_number-1)]):
                        synonym.append(l.name())
                    #if l.antonyms():
                    #    antonyms.append(l.antonyms()[0].name())
        
        if (synonym) :
            print(word, set(synonym))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phamduy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


Executes and writes portions of testing plans, protocols, and documentation for assigned portion of application; identifies and debugs issues with code and suggests changes or improvements.
portions {'portion'}
plans {'plan'}
portion {'component'}
issues {'issue'}
suggests {'suggest'}
changes {'change'}
improvements {'improvement'}
Maintain Network Performance by assisting with configurations and connections
configurations {'configuration'}
Supports the regional compliance manager with day-to-day compliance support to assist the business and other stakeholders.
manager {'director'}
support {'supporting'}
Keep up to date with local and national business and economic issues
issues {'issue'}
Assist with Service Organization Control (SOC) examinations
Establish a high level of code quality by writing unit tests, participating in code reviews, reducing cyclomatic complexity, removing code duplication, and debugging software modules
tests {'test'}
participating {'participate'}
reviews {'revi

Work with clients to develop network engineering solutions from concept through implementation
clients {'client'}
engineering {'technology'}
solutions {'solution'}
concept {'conception'}
Work across teams to report on key metrics and develop KPIs
teams {'team'}
metrics {'metric'}
Reporting to the Project Manager based in San Diego, CA.
Developing novel techniques to tackle the challenges of prediction, optimization, and classification as they relate to: pattern recognition, target detection, tracking, reasoning and inferencing systems.
techniques {'technique'}
challenges {'challenge'}
prediction {'forecasting'}
classification {'categorization'}
recognition {'acknowledgment', 'identification'}
systems {'system'}
Collaborate with technical and non-technical users on software implementation.
Follow coding standards and best practices
standards {'standard'}
best {'good', 'better'}
practices {'practice'}
Produce data visualizations that provide insight into data set structure and meaning
vi

services {'service'}
services {'service'}
Provide leadership at conferences and actively participate at events and meet-ups
conferences {'conference'}
events {'event'}
The role involves model construction, information extraction, prediction and finding solutions to NP Hard problems for mission critical applications using large datasets derived from a broad spectrum of domains for the following sectors - logistics, consumer goods, capital markets, and telecommunications.
role {'part'}
prediction {'forecasting'}
solutions {'solution'}
problems {'problem'}
applications {'application'}
derived {'derive'}
broad {'extensive', 'wide'}
domains {'domain'}
sectors {'sector'}
markets {'market', 'marketplace'}
Responsible for cleaning, transforming, enriching, and analyzing vast amounts of raw data from various systems using Apache Spark and other analytics packages to develop valuable features and to provide ready-to-use data to a variety of stakeholders for robust downstream analysis.
cleaning {

Embedded SW Development for Automotive Products (They use a proprietary OS and OSEK OS, so it does not matter if the engineer has linux or windows experience), hands on knowledge on C or C++ programming
use {'utilize'}
hands {'hand'}
Design, prototype, and implementation of new solutions
new {'fresh'}
solutions {'solution'}
Leverage data and requirements gathered from business partners to build machine learning applications
requirements {'requirement'}
partners {'partner'}
applications {'application'}
Interpret, confirm, and/or challenge business requirements and document technical requirements
requirements {'requirement'}
requirements {'requirement'}
Collaborate with cross-functional engineering teams to develop new platform features
engineering {'technology'}
teams {'team'}
new {'fresh'}
features {'feature'}
Work closely with engineering team to deliver machine learning models into production
engineering {'technology'}
models {'modelling', 'modeling', 'model'}
This position is not wi

...

operations {'operation'}
teams {'team'}
solutions {'solution'}
meet {'fulfill', 'satisfy'}
Establish best practices for highly scalable and available infrastructure
best {'good', 'better'}
practices {'practice'}
highly {'extremely'}
Position 2: .NET app dev and SQL--moving across whole app CRUD and general configurations, Backend focused
configurations {'configuration'}
focused {'focus'}
Assess / deliver on talent needs for AI/ML success
needs {'need'}
Build and maintain a scalable multi-stage CI system. The CI system must be able to support a wide range of teams ranging from embedded software team, Windows GUI software team to teams developing software for the cloud and machine learning
wide {'broad'}
teams {'team'}
teams {'team'}
Develop skills and knowledge by continuing education on topics related to the position responsibilities
skills {'skill'}
topics {'topic'}
position {'positioning'}
responsibilities {'responsibility'}
Maintains accountability for adherence to company and divis

applications {'application'}
Create and maintain strong professional relationships with all ATOM stakeholders
relationships {'relationship'}
Perform analysis and software studies and evaluate alternative and recommend solutions
evaluate {'assess'}
solutions {'solution'}
Reports to and takes direction from the Presales Head
takes {'take'}
Design verification and validation of system
verification {'confirmation'}
Train end users on the new reports and dashboards.
end {'close'}
new {'fresh'}
reports {'report'}
dashboards {'dashboard'}
Supports the business with reviewing submitted requests for the entertainment of third parties.
requests {'request'}
parties {'party'}
Analyze customer wireless network technical roadmap and propose strategic planning
planning {'preparation'}
Gain exposure to current technologies such as containerization and cloud
technologies {'technology'}
Builds and maintains a robust library of reusable algorithms and supporting code such that research efforts are based 

Developing embedded software units in an Agile environment with test-driven development (TDD) methodology.
units {'unit'}
Learn our domain of research and clinical applications of flow cytometry.
applications {'application'}
Contributes to the growth of the analytic community.
growth {'growing', 'development'}
analytic {'analytical'}
Work on optimizing the performance of the data integration platforms
integration {'integrating'}
platforms {'platform'}
Works closely with software developers to create software artifacts including test plans, test cases, test procedures and test reports
developers {'developer'}
create {'produce'}
plans {'plan'}
cases {'case'}
procedures {'procedure'}
reports {'report'}
Maintains network performance by performing network monitoring and analysis, and performance tuning; troubleshooting network problems; escalating problems to vendor.
troubleshooting {'troubleshoot'}
problems {'problem'}
escalating {'escalate'}
problems {'problem'}
Supporting vehicle systems

advisory {'consultative'}
services {'service'}
support {'supporting'}
program {'curriculum'}
Collaborate with data engineers to optimize the data platform to produce the analysis results in the quickest way possible
engineers {'engineer'}
results {'result'}
quickest {'immediate'}
way {'manner', 'means'}
possible {'potential'}
Conduct relevant research, data analysis, and create reports
create {'produce'}
reports {'report'}
Build causal and predictive models utilizing statistical programming languages such as R based on integrated data of various structures from numerous sources stored in a SQL environment.
models {'modelling', 'modeling', 'model'}
languages {'language'}
integrated {'incorporated'}
various {'several'}
structures {'structure'}
sources {'source'}
Quickly understands the business issues and data challenges of client's organization and industry
understands {'understand'}
issues {'issue'}
challenges {'challenge'}
client {'customer'}
Rather than answering questions directly, 

KeyError: 26947